This Notebook evaluates test cases for the 1D Shock-Acoustic-Wave interaction problem as presented presented in the phd thesis "Implicit Discontinuous Galerkin Shock Tracking methods for Compressible Flows with Shocks" (Vandergrift 2024).

In [1]:
#r ".\binaries\BoSSSpad.dll"
#r ".\binaries\XESTSF.dll"
using System;
using BoSSS.Foundation;
using BoSSS.Application.BoSSSpad;
using BoSSS.Solution;
using BoSSS.Solution.Tecplot;
using BoSSS.Foundation.IO;
using ilPSP.Tracing;
using BoSSS.Solution.Utils;
using ilPSP.LinSolvers;
using BoSSS.Solution.NSECommon;
using ilPSP.Connectors.Matlab;
using ilPSP;
using BoSSS.Foundation.Grid.Classic;
using ilPSP.Utils;
using BoSSS.Foundation.Grid.RefElements;
using System.Collections.Generic;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.XDG;
using BoSSS.Solution.XdgTimestepping;
using BoSSS.Solution.AdvancedSolvers;
using System.Linq;
using BoSSS.Foundation.Grid.Aggregation;
using BoSSS.Platform;
using MPI.Wrappers;
using System.Diagnostics;
using System.IO;
using System.Collections;
using BoSSS.Foundation.Quadrature;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Gnuplot;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using ApplicationWithIDT;
using XESTSF;
using XDGShock;
Init();


In [2]:
var ass= new MatrixAssembler(1);
BoSSSshell.WorkflowMgm.Init("XESTSF_ShockAcousticInteraction1D");
BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation();
BoSSSshell.WorkflowMgm.DefaultDatabase


{ Session Count = 6; Grid Count = 6; Path = \\dc3\userspace\sebastian\cluster\XESTSF_ShockAcousticInteraction1D }

In [3]:
string dbPath = BoSSSshell.WorkflowMgm.DefaultDatabase.Path; 
dbPath

\\dc3\userspace\sebastian\cluster\XESTSF_ShockAcousticInteraction1D

In [4]:
(double[] xVals, double[]yVals) GetPlot(double xMin,double xMax, double t, int nRef,string field,ISessionInfo si){
//double shockPosition=0.5;
var p1= new double[] {xMin,t};
var p2=new double[] {xMax,t};

//timesteps to compare
var tiEnd=si.Timesteps.Last();

// # use p_per from BoSSS 
var pEnd =tiEnd.GetField(field);
double[] yVals=pEnd.EvaluateAlongLine(p1,p2,nRef);
double[] xVals= GenericBlas.Linspace(xMin,xMax,nRef);
return(xVals,yVals);
}

In [5]:
var db=OpenOrCreateDatabase(dbPath);
var ses=db.Sessions;
ses

#0: XESTSF_AcousticWave1D	AW_p3_xCells61_tCells61_tMax4_bump_Mach1.5_sP1.5_wP0.3_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.4	3/22/2024 8:38:21 PM	f6b1e3da...
#1: XESTSF_AcousticWave1D	AW_p3_xCells61_tCells61_tMax4_1sinus_Mach1.5_sP1.5_wP0.3_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8	3/22/2024 8:13:06 PM	53f8b03c...
#2: XESTSF_AcousticWave1D	AW_p3_xCells61_tCells61_tMax1.5_bump_Mach1.5_sP1.5_wP0_ampneg0_amppos1E-05_wLTrue_wSTrue_wL0.4	3/22/2024 8:38:00 PM	4eedd833...
#3: XESTSF_AcousticWave1D	AW_p3_xCells61_tCells61_tMax4_bump_Mach1.5_sP0.5_wP0.9_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.4	3/22/2024 8:37:42 PM	e4f163a8...
#4: XESTSF_AcousticWave1D	AW_p3_xCells61_tCells61_tMax4_1sinus_Mach1.5_sP0.5_wP0.9_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8	3/22/2024 8:12:18 PM	2e01f579...
#5: XESTSF_AcousticWave1D	AW_p3_xCells61_tCells61_tMax1.5_1sinus_Mach1.5_sP1.5_wP0_ampneg0_amppos1E-05_wLTrue_wSTrue_wL0.8	3/22/2024 8:12:42 PM	1846d0aa...


In [8]:
var si=db.Sessions.Pick(0);
var ti=si.Timesteps.Last();
//si.Timesteps
si

XESTSF_AcousticWave1D	AW_p3_xCells61_tCells61_tMax4_bump_Mach1.5_sP1.5_wP0.3_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.4	3/22/2024 8:38:21 PM	f6b1e3da...

# Plotting

define general params for plotting (must substract some eps because, otherwise some points are not located in the space-time grid)

In [21]:
var eps=0.000001;
double xMin=0 +eps;double xMax=3.0-eps; int xRef=100; 
double tMin =0 + eps;int tRef=100;
int nRows=2;

obtain tMax from session name

In [22]:
// Format
var fmt= new PlotFormat();
fmt.WithPointSize(0.0);
fmt.LineColor=LineColors.Red;

## Pressure Pertubations

In [23]:
using System.Text.RegularExpressions;
foreach(var si in ses){
    string pattern = @"tMax(\d+)";
    Match match = Regex.Match(si.Name, pattern);
    double tMax = 1.5;
    try{
        tMax = double.Parse(match.Groups[1].Value)-eps;
    }catch{
    }
    Console.WriteLine($"doing:{si.Name}; tMax:{tMax}");
    string field="p_per";
    string csvFileName = si.Name + "_p_per.csv";
    Plot2Ddata[,] multiplotsPressure = new Plot2Ddata[(tRef+1)/nRows,nRows];
    using (StreamWriter writer = new StreamWriter(csvFileName))
            {
            for(int Ti=0;Ti< tRef;Ti++){
                var plot= new Plot2Ddata();
                plot.Xlabel="x";
                plot.Ylabel="p'";
                double t=((double) tRef-1-Ti)/((double) tRef-1)*tMin+((double) Ti)/(tRef-1)*tMax;
                //Console.WriteLine(t);
                (double[] xVals,double[] yVals)=GetPlot(xMin,xMax,t,xRef,field, si);
                plot.AddDataGroup("t="+Math.Round(t,2),xVals,yVals,fmt);
                multiplotsPressure[Ti/nRows,Ti %nRows]=plot;
                if (Ti == 0)
                {
                    // Write header row
                    writer.WriteLine($"x/t,{string.Join(",", xVals)}");
                }
                // Write data row
                writer.WriteLine($"{Math.Round(t, 2)},{string.Join(",", yVals)}");
                        
            }
    };
}



In [12]:
//Array.ForEach(multiplotsPressure.Cast<Plot2Ddata>().ToArray(), 
//    plot => { plot.YrangeMin = -1.5 * 1e-5; plot.YrangeMax = 1.5 * 1e-5; });
// multiplotsPressure[1,1].YrangeMax = 5 * 1e-7; multiplotsPressure[1,1].YrangeMin = -6 * 1e-7;
// multiplotsPressure[2,0].YrangeMax = 5 * 1e-7; multiplotsPressure[2,0].YrangeMin = -6 * 1e-7;
// multiplotsPressure[2,1].YrangeMax = 5 * 1e-7; multiplotsPressure[2,1].YrangeMin = -6 * 1e-7;
//multiplotsPressure.ToGnuplot().PlotSVG(xRes:900,yRes:600)

## Density Pertubations

In [ ]:
foreach(var si in ses){
        string pattern = @"tMax(\d+)";
    Match match = Regex.Match(si.Name, pattern);
    double tMax = 1.5;
    try{
        tMax = double.Parse(match.Groups[1].Value)-eps;
    }catch{
    }
    Console.WriteLine($"doing:{si.Name}; tMax:{tMax}");
    string field="rho_per";
    // Format
    var fmt= new PlotFormat();
    fmt.WithPointSize(0.0);
    fmt.LineColor=LineColors.Red;

    Plot2Ddata[,] multiplotsDensity = new Plot2Ddata[(tRef+1)/nRows,nRows];
    string csvFileName = si.Name + "_rho_per.csv";
    using (StreamWriter writer = new StreamWriter(csvFileName)){
        for(int Ti=0;Ti< tRef;Ti++){
            var plot= new Plot2Ddata();
            plot.Xlabel="x";
            plot.Ylabel="rho'";
            double t=((double) tRef-1-Ti)/((double) tRef-1)*tMin+((double) Ti)/(tRef-1)*tMax;
            //Console.WriteLine(t);
            (double[] xVals,double[] yVals)=GetPlot(xMin,xMax,t,xRef,field, si);
            plot.AddDataGroup("t="+Math.Round(t,2),xVals,yVals,fmt);
            multiplotsDensity[Ti/nRows,Ti %nRows]=plot;
            if (Ti == 0)
            {
                // Write header row
                writer.WriteLine($"x/t,{string.Join(",", xVals)}");
            }
            // Write data row
            writer.WriteLine($"{Math.Round(t, 2)},{string.Join(",", yVals)}");
                    
        }
        
    };                                              
}

In [14]:
// Array.ForEach(multiplotsDensity.Cast<Plot2Ddata>().ToArray(), 
//     plot => { plot.YrangeMin = -1 * 1e-5; plot.YrangeMax = 1.5 * 1e-5; });
// // multiplotsDensity[1,1].YrangeMax = 5 * 1e-7; multiplotsDensity[1,1].YrangeMin = -6 * 1e-7;
// //multiplotsDensity[2,0].YrangeMax = 5 * 1e-7; multiplotsDensity[2,0].YrangeMin = -6 * 1e-7;
// //multiplotsDensity[2,1].YrangeMax = 5 * 1e-7; multiplotsDensity[2,1].YrangeMin = -6 * 1e-7;
// multiplotsDensity.ToGnuplot().PlotSVG(xRes:1000,yRes:600)

In [16]:
(double[] xVals, double[]yVals) GetPlotLevelSet(double tMin,double tMax, int nRef,ISessionInfo si){
//double shockPosition=0.5;
var p1= new double[] {0.0,tMin};
var p2=new double[] {0.0,tMax};

//timesteps to compare
var tiEnd=si.Timesteps.Last();
// # use p_per from BoSSS 
var pEnd =tiEnd.GetField("LevelSet");
double[] yVals=pEnd.EvaluateAlongLine(p1,p2,nRef);
double[] xVals= GenericBlas.Linspace(xMin,xMax,nRef);
return(xVals,yVals);
}

In [17]:
// Format
var fmt= new PlotFormat();
fmt.WithPointSize(0.0);
fmt.LineColor=LineColors.Red;
string csvFileName = "XESTSFLevelSetPertubations.csv";
var plotLevelSet= new Plot2Ddata();
using (StreamWriter writer = new StreamWriter(csvFileName)){
    
    //Console.WriteLine(t);
    (double[] xVals,double[] yVals)=GetPlotLevelSet(tMin,tMax,1000, si);
    
    plotLevelSet.AddDataGroup("LevelSet",xVals,yVals,fmt);
        // Write header row
    writer.WriteLine($"t,{string.Join(",", xVals)}");
    
    // Write data row
    writer.WriteLine($"phi(t),{string.Join(",", yVals)}");
            
}

plotLevelSet.Ylabel="\varphi(0,t)";
plotLevelSet.ToGnuplot().PlotSVG(xRes:800,yRes:800)  


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -1.500000 
 
 
 
 
 -1.500000 
 
 
 
 
 -1.500000 
 
 
 
 
 -1.499999 
 
 
 
 
 -1.499999 
 
 
 
 
 -1.499999 
 
 
 
 
 -1.499999 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 2.5 
 
 
 
 
 3 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 arphi(0,t) 
 
 
 
 
 LevelSet 
 
 
 LevelSet 
 
 
 
	<path stroke='rgb(255, 0, 0)' d='M588.5,57.1 L641.9,57.1 M113.4,764.0 L114.0,763.8 L114.7,763.6 L115.3,763.4 L115.9,763.3 L116.6,763.1
 L117.2,762.9 L117.9,762.7 L118.5,762.5 L119.1,762.3 L119.8,762.1 L120.4,762.0 L121.0,761.8 L121.7,761.6
 L122.3,761.4 L123.0,761.2 L123.6,761.0 L124.2,761.0 L124.9,761.0 L125.5,761.0 L126.1,761.0 L126.8,761.0
 L127.4,761.0 L128.1,761.0 L128.7,761.0 L129.3,761.0 L130.0,761.0 L130.6,761.0 L131.2,761.0 L131.9,761.0
 L132.5,761.0 L133.2,761.0 L133.8,761.0 L134.4,761.0 L135.1,761.0 L135.7,761.0 L136.3,761.0 L137.0,761.0
 L137.6,761.0 L138.3,761.0 L138.9,761.0 L139.5,761.0 L140.2,761.0 L140.8,761.0 L141.4,761.0 L142.1,761.0
 L142.7,761.0 L143.4,761.0 L144.0,761.0 L144.6,761.0 L145.3,761.0 L145.9,761.0 L146.5,761.0 L147.2,761.0
 L147.8,761.0 L148.5,761.0 L149.1,761.0 L149.7,761.0 L150.4,761.0 L151.0,761.0 L151.6,761.0 L152.3,761.0
 L152.9,761.0 L153.6,761.0 L154.2,761.0 L154.8,761.0 L155.5,761.0 L156.1,761.0 L156.7,761.0 L157.4,761.0
 L158.0,761.0 L158.7,761.0 L159.3,761.0 L159.9,761.0 L160.6,761.0 L161.2,761.0 L161.8,761.0 L162.5,761.0
 L163.1,761.0 L163.7,761.0 L164.4,761.0 L165.0,761.0 L165.7,761.0 L166.3,761.0 L166.9,761.0 L167.6,761.0
 L168.2,761.0 L168.8,761.0 L169.5,761.0 L170.1,761.0 L170.8,761.0 L171.4,761.0 L172.0,761.0 L172.7,761.0
 L173.3,761.0 L173.9,761.0 L174.6,761.0 L175.2,761.0 L175.9,761.0 L176.5,761.0 L177.1,761.0 L177.8,761.0
 L178.4,761.0 L179.0,761.0 L179.7,761.0 L180.3,761.0 L181.0,761.0 L181.6,761.0 L182.2,761.0 L182.9,761.0
 L183.5,761.0 L184.1,761.0 L184.8,761.0 L185.4,761.0 L186.1,761.0 L186.7,761.0 L187.3,761.0 L188.0,761.0
 L188.6,761.0 L189.2,761.0 L189.9,761.0 L190.5,761.0 L191.2,761.0 L191.8,761.0 L192.4,761.0 L193.1,761.0
 L193.7,761.0 L194.3,761.0 L195.0,761.0 L195.6,761.0 L196.3,761.0 L196.9,761.0 L197.5,761.0 L198.2,761.0
 L198.8,761.0 L199.4,761.0 L200.1,761.0 L200.7,761.0 L201.4,761.0 L202.0,761.0 L202.6,761.0 L203.3,761.0
 L203.9,761.0 L204.5,761.0 L205.2,761.0 L205.8,761.0 L206.5,761.0 L207.1,761.0 L207.7,761.0 L208.4,761.0
 L209.0,761.0 L209.6,761.0 L210.3,761.0 L210.9,761.0 L211.6,761.0 L212.2,761.0 L212.8,761.0 L213.5,761.0
 L214.1,761.0 L214.7,761.0 L215.4,761.0 L216.0,761.0 L216.6,761.0 L217.3,761.0 L217.9,761.0 L218.6,761.0
 L219.2,761.0 L219.8,761.0 L220.5,761.0 L221.1,761.0 L221.7,761.0 L222.4,761.0 L223.0,761.0 L223.7,761.0
 L224.3,761.0 L224.9,761.0 L225.6,761.0 L226.2,761.0 L226.8,761.0 L227.5,761.0 L228.1,761.0 L228.8,761.0
 L229.4,761.0 L230.0,761.0 L230.7,761.0 L231.3,761.0 L231.9,761.0 L232.6,761.0 L233.2,761.0 L233.9,761.0
 L234.5,761.0 L235.1,761.0 L235.8,761.0 L236.4,761.0 L237.0,761.0 L237.7,761.0 L238.3,761.0 L239.0,761.0
 L239.6,761.0 L240.2,761.0 L240.9,761.0 L241.5,761.0 L242.1,761.0 L242.8,761.0 L243.4,761.0 L244.1,761.0
 L244.7,761.0 L245.3,761.0 L246.0,761.0 L246.6,761.0 L247.2,761.0 L247.9,761.0 L248.5,761.0 L249.2,761.0
 L249.8,761.0 L250.4,761.0 L251.1,761.0 L251.7,761.0 L252.3,761.0 L253.0,761.0 L253.6,761.0 L254.3,761.0
 L254.9,761.0 L255.5,761.0 L256.2,760.9 L256.8,760.9 L257.4,760.9 L258.1,760.9 L258.7,760.9 L259.4,760.9
 L260.0,760.9 L260.6,760.9 L261.3,760.9 L261.9,761.0 L262.5,761.0 L263.2,761.0 L263.8,761.0 L264.4,761.0
 L265.1,761.0 L265.7,761.0 L266.4,761.0 L267.0,761.0 L267.6,761.0 L268.3,761.0 L268.9,761.0 L269.5,761.0
 L270.2,